In [1]:
import pandas as pd
import io
import sqlalchemy as sa
import urllib


from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

SERVER = os.environ.get("SERVER")
DATABASE = os.environ.get("DATABASE")

USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")

In [53]:
conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
query = """
SELECT  [KodStacji]
      ,[KodMiedzynarodowy]
      ,[NazwaStacji]
      ,[StaryKodStacji]
      ,[DataUruchomienia]
      ,[DataZamkniecia]
      ,[TypStacji]
      ,[TypObszaru]
      ,[RodzajStacji]
      ,[Wojewodztwo]
      ,[Miejscowosc]
      ,[Ulica]
      ,[Latitude]
      ,[Longitude]
  FROM [SmogoliczkaArchive].[dbo].[stacje]
"""

In [60]:

geolocator = Nominatim(user_agent="psierkin@gmail.com")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_orig = pd.read_sql(query, engine)
df=df_orig.copy()
df['adres']=df['Ulica']+", "+df['Miejscowosc']
mask=df['Longitude']<0
df=df[mask]

df['lokalizacja'] = df['adres'].apply(lambda row: geocode(row))
df['lokalizacja_2'] = df['Miejscowosc'].apply(lambda row: geocode(row))
df['lokalizacja'].fillna(df['lokalizacja_2'],inplace=True)
df['punkt'] = df['lokalizacja'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude_new', 'longitude_new', 'altitude']] = pd.DataFrame(df['punkt'].tolist(), index=df.index)



df['Latitude'] = df['Latitude'].replace(-999, np.nan)
df['Longitude'] = df['Longitude'].replace(-999, np.nan)
df['Latitude'].fillna(df['latitude_new'],inplace=True)
df['Longitude'].fillna(df['longitude_new'],inplace=True)
df.drop(columns=['adres','lokalizacja','punkt','latitude_new','lokalizacja_2', 'longitude_new', 'altitude'],inplace=True)
df_orig.update(df)

In [62]:
df_orig[df_orig['Longitude']<0]

,KodStacji,KodMiedzynarodowy,NazwaStacji,StaryKodStacji,DataUruchomienia,DataZamkniecia,TypStacji,TypObszaru,RodzajStacji,Wojewodztwo,Miejscowosc,Ulica,Latitude,Longitude


In [64]:
#send data back to sql
df_orig.to_sql("stacje_v2", engine, if_exists="replace",index=False)